In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Load and visualize the data
import numpy as np
import pandas as pd

data=pd.read_csv('/kaggle/input/imdb-movie-review-dataset/movie_data.csv')
    

In [ ]:

import re

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower())
    text = text + " ".join(emoticons).replace('-', '')
    return text

preprocessor("</a>This :) is :( a test :-)!")

In [ ]:
data['review'] = data['review'].apply(preprocessor)



In [ ]:
data['review'][3]

In [ ]:
# split the words in the review
reviews=[]
for i in range(data['review'].shape[0]):
    words=data['review'][i].split()
    reviews.append(words)

In [ ]:
reviews[0]

In [ ]:
# Now I want to join all the words in whole review
words=[]
for i in range(data['review'].shape[0]):
    word=[word for word in data['review'][i].split()]
    words+=word
    
    

In [ ]:
len(words)

## Encoding the words
The embedding lookup requires that we pass in integers to our network. The easiest way to do this is to create dictionaries that map the words in the vocabulary to integers. Then we can convert each of our reviews into integers so they can be passed into the network.

>> Now I'm going to encode the words with integers. Build a dictionary that maps words to integers. Later I'm going to pad our input vectors with zeros, so make sure the integers start at 1, not 0. Also, convert the reviews to integers and store the reviews in a new list called reviews_ints.

In [ ]:
from collections import Counter

# build a dictionary that maps words to integers
counts=Counter(words)
vocab=sorted(counts,key=counts.get,reverse=True)
vocab_to_int={word:ii for ii,word in enumerate(vocab,start=1)}


## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints

reviews_ints=[]

for review in reviews:
    reviews_ints.append([vocab_to_int[word] for word in review])

## Test the code

As a text that implemented the dictionary correctly, print out the number of unique words in your vocabulary and the contents of the first, tokenized review.

In [ ]:
# stats about vocabulary
print("Unique words: ",len(vocab_to_int))
print()

# print tokens in first review
print('Tokenized review : \n',reviews_ints[:1])

In [ ]:
# getting the labels corresponding to the Reviews 0="Negative" and 1="Positive"
labels=data['sentiment'].to_numpy()
len(labels)

## Removing Outliers
As an additional pre-processing step, we want to make sure that our reviews are in good shape for standard processing. That is, our network will expect a standard input text size, and so, we'll want to shape our reviews into a specific length. We'll approach this task in two main steps:

1. Getting rid of extremely long or short reviews; the outliers
2. Padding/truncating the remaining data so that we have reviews of the same length.
Before we pad our review text, we should check for reviews of extremely short or long lengths; outliers that may mess with our training.

In [ ]:
# outlier review stats

review_lens=Counter([len(x) for x in reviews_ints])
print("Zero Length Reviews : {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

# there is no empty review
# The review with maximum length is 2498 

Okay, there is issues here. And, the maximum review length is way too many steps (2498) for our RNN. We'll have to remove any super short reviews and truncate super long reviews. This removes outliers and should allow our model to train more efficiently.

First, remove any reviews with zero length from the reviews_ints list and their corresponding label in encoded_labels.

## Padding sequences
To deal with both short and very long reviews, we'll pad or truncate all our reviews to a specific length. For reviews shorter than some seq_length, we'll pad with 0s. For reviews longer than seq_length, we can truncate them to the first seq_length words. A good seq_length, in this case, is 200.

>>Define a function that returns an array features that contains the padded data, of a standard size, that we'll pass to the network.

>>* The data should come from review_ints, since we want to feed integers to the network.
>>* Each row should be seq_length elements long.
>>* For reviews shorter than seq_length words, left pad with 0s. That is, if the review is ['best', 'movie', 'ever'], [117, 18, 128] as integers, the row will look like [0, 0, 0, ..., 0, 117, 18, 128].
>>* For reviews longer than seq_length, use only the first seq_length words as the feature vector.
As a small example, if the seq_length=10 and an input review is:

[117, 18, 128]  
The resultant, padded sequence should be:

[0, 0, 0, 0, 0, 0, 0, 117, 18, 128]  
Your final features array should be a 2D array, with as many rows as there are reviews, and as many columns as the specified seq_length.

In [ ]:
def pad_features(reviews_ints,seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    
    # getting the correct rows x cols shape
    features=np.zeros((len(reviews_ints),seq_length),dtype=int)
    
    # For each review I grab that review and
    for i,row in enumerate(reviews_ints):
        features[i,-len(row):]=np.array(row)[:seq_length]
        
    return features

In [ ]:
# test The Implementation

seq_length=200

features=pad_features(reviews_ints,seq_length=seq_length)

# test Statements
assert len(features)==len(reviews_ints),"Features should have as many rows as reviews"

assert len(features[0])==seq_length,"Each Features row should contains Sequence length values"

# Print first 10 values of first 30 batches
print(features[:30,:10])

## Training, Validation, Test
With our data in nice shape, we'll split it into training, validation, and test sets.

>>Create the training, validation, and test sets.

>>* You'll need to create sets for the features and the labels, train_x and train_y, for example.  
>>* Define a split fraction, split_frac as the fraction of data to keep in the training set. Usually this is set to 0.8 or 0.9.
>>* Whatever data is left will be split in half to create the validation and testing data.


In [ ]:
features[0]

In [ ]:
split_frac=0.8


### split data into training, validation, and test data (features and labels, x and y)

split_idx=int(len(features)*split_frac)

train_x,remaining_x=features[:split_idx],features[split_idx:]
train_y,remaining_y=labels[:split_idx],labels[split_idx:]

test_idx=int(len(remaining_x)*0.5)
val_x,test_x=remaining_x[:test_idx],remaining_x[test_idx:]
val_y,test_y=remaining_y[:test_idx],remaining_x[test_idx:]

## print out the shapes of your resultant feature data

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

## DataLoaders and Batching
After creating training, test, and validation data, we can create DataLoaders for this data by following two steps:

>>1. Create a known format for accessing our data, using TensorDataset which takes in an input set of data and a target set of data with the same first dimension, and creates a dataset.
>>2. Create DataLoaders and batch our training, validation, and test Tensor datasets. 

>>**train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
train_loader = DataLoader(train_data, batch_size=batch_size)**  


This is an alternative to creating a generator function for batching our data into full batches.

In [ ]:
import torch

from torch.utils.data import TensorDataset,DataLoader

# Create Tensor dataset
train_data=TensorDataset(torch.from_numpy(train_x),torch.from_numpy(train_y))
val_data=TensorDataset(torch.from_numpy(val_x),torch.from_numpy(val_y))
test_data=TensorDataset(torch.from_numpy(test_x),torch.from_numpy(test_y))

# Dataloader
batch_size=50

# Make sure to shuffle your training data

train_loader=DataLoader(train_data,shuffle=True,batch_size=batch_size,drop_last=True)
valid_loader=DataLoader(val_data,shuffle=True,batch_size=batch_size,drop_last=True)
test_loader=DataLoader(test_data,shuffle=True,batch_size=batch_size,drop_last=True)


In [ ]:
# Obtain One batch of Training Data
dataiter=iter(train_loader)
sample_x,sample_y=dataiter.next()

print("Sample Input Size: ",sample_x.size()) # batch_size,seq_length
print("Sample Input :",sample_x)
print()

print("Sample Label size :",sample_y.size()) # Batch_size
print("Sample Label :",sample_y)

## IMDB REVIEW Network with PyTorch




The layers are as follows:

>>1. An embedding layer that converts our word tokens (integers) into embeddings of a specific size.
>>2. An LSTM layer defined by a hidden_state size and number of layers
>>3. A fully-connected output layer that maps the LSTM layer outputs to a desired output_size
>>4. A sigmoid activation layer which turns all outputs into a value 0-1; return only the last sigmoid output as the output of this network.

### The Embedding Layer
We need to add an embedding layer because there are 105225 words in our vocabulary. It is massively inefficient to one-hot encode that many classes. So, instead of one-hot encoding, we can have an embedding layer and use that layer as a lookup table. You could train an embedding layer using Word2Vec, then load it here. But, it's fine to just make a new layer, using it for only dimensionality reduction, and let the network learn the weights.

### The LSTM Layer(s)
We'll create an LSTM to use in our recurrent network, which takes in an input_size, a hidden_dim, a number of layers, a dropout probability (for dropout between multiple layers), and a batch_first parameter.

Most of the time, you're network will have better performance with more layers; between 2-3. Adding more layers allows the network to learn really complex relationships.

In [ ]:
# First Checking if GPU is Available

train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print("Training on GPU")

else:
    print("Training on CPU")

In [ ]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
        

## Instantiate the network
Here, we'll instantiate the network. First up, defining the hyperparameters.

>>* vocab_size: Size of our vocabulary or the range of values for our input, word tokens.
>>* output_size: Size of our desired output; the number of class scores we want to output (pos/neg).
>>* embedding_dim: Number of columns in the embedding lookup table; size of our embeddings.
>>* hidden_dim: Number of units in the hidden layers of our LSTM cells. Usually larger is better performance wise. Common values are 128, 256, 512, etc.
>>* n_layers: Number of LSTM layers in the network. Typically between 1-3


In [ ]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

---
## Training

Below is the typical training code. If you want to do this yourself, feel free to delete all this code and implement it yourself. You can also add code to save a model by name.

>We'll also be using a new kind of cross entropy loss, which is designed to work with a single Sigmoid output. [BCELoss](https://pytorch.org/docs/stable/nn.html#bceloss), or **Binary Cross Entropy Loss**, applies cross entropy loss to a single value between 0 and 1.

We also have some data and training hyparameters:

* `lr`: Learning rate for our optimizer.
* `epochs`: Number of times to iterate through the training dataset.
* `clip`: The maximum gradient value to clip at (to prevent exploding gradients).

In [ ]:
# Loss and optimization functions
lr=0.001
criterion=nn.BCELoss()
optimizer=torch.optim.Adam(net.parameters(),lr=lr)

In [ ]:
# training params

epochs = 4 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

---
## Testing

There are a few ways to test your network.

* **Test data performance:** First, we'll see how our trained model performs on all of our defined test_data, above. We'll calculate the average loss and accuracy over the test data.

* **Inference on user-generated data:** Second, we'll see if we can input just one example review at a time (without a label), and see what the trained model predicts. Looking at new, user input data like this, and predicting an output label, is called **inference**.

In [ ]:
# Get test data loss and accuracy

test_losses=[]
num_correct=0

# init hidden state

h=net.init_hidden(batch_size)

net.eval()

# Iterate Over test data
for inputs,lebels in test_loader:
    
    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    
    h=tuple([each.data for each in h])
    
    if(train_on_gpu):
        inputs,labels=inputs.cuda(),labels.cuda()
        
    # Get predicted outputs
    output,h=net(inputs,h)
    
    # calculate loss
    test_loss=criterion(output.squeeze(),labels.float())
    test_losses.append(test_loss.item())
    
    # Convert output probabilities to predicted class(0 or 1) 
    pred=torch.round(output.squeeze())   # rounds to the nearest integer
    
    # Compare predictions to true label
    
    correct_tensor=pred.eq(labels.float().view_as(pred))
    correct=np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct+=np.sum(correct)
    
    
    
    
#stats

# Average test loss
print("Test Loss: {:.3f}".format(np.mean(test_losses)))

# Accuracy over all test data
test_acc=num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))    
    
    
    
    

### Inference on a test review

You can change this test_review to any text that you want. Read it and think: is it pos or neg? Then see if your model predicts correctly!
    
> Write a `predict` function that takes in a trained net, a plain text_review, and a sequence length, and prints out a custom statement for a positive or negative review!
* You can use any functions that you've already defined or define any helper functions you want to complete `predict`, but it should just take in a trained net, a text review, and a sequence length.


In [ ]:
# negative test review
test_review_neg='The worst movie I have seen; acting was terrible and I want my money back. This movie had bad acting and the dialogue was slow.'

In [ ]:
from string import punctuation

def tokenize_review(test_review):
    test_review=test_review.lower() # Lowercase
    # Get Rid of punctuation
    test_text=''.join(c for c in test_review if c not in punctuation)
    
    # Splitting by spaces
    test_words=test_text.split()
    
    # Tokens
    test_ints=[]
    test_ints.append([vocab_to_int[word] for word in test_words])
    
    
    return test_ints

# test code and generate tokenized review

test_ints=tokenize_review(test_review_neg)
print(test_ints)

In [ ]:
# Test sequence Padding
seq_length=200
features=pad_features(test_ints,seq_length)
print(features)

In [ ]:
# test conversion to tensor and pass into your model
feature_tensor=torch.from_numpy(features)
print(feature_tensor.size())

In [ ]:
def predict(net,test_review,sequence_length=200):
    net.eval()
    
    # Tokenize Review
    test_ints=tokenize_review(test_review)
    
    # pad tokenized sequence
    seq_length=sequence_length
    features=pad_features(test_ints,seq_length)
    
    # Convert to tensor to pass into your model
    feature_tensor=torch.from_numpy(features)
    batch_size=feature_tensor.size(0)
    
    # Initialize hidden State
    h=net.init_hidden(batch_size)
    
    if(train_on_gpu):
        feature_tensor=feature_tensor.cuda()
        
    # Get the output from the model
    output,h=net(feature_tensor,h)
    
    # convert output probabilities to predicted class (0 or 1)
    pred=torch.round(output.squeeze())
    #printing output value, before rounding
    print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))
    
    
    # Print custom response
    if(pred.item()==1):
        print("Positive Review Detected !")
        
    else:
        print("Negative Review Detected !")
    

In [ ]:
# positive test review
test_review_pos = 'This movie had the best acting and the dialogue was so good. I loved it.'


In [ ]:
# call function
seq_length=200 # good to use the length that was trained on

predict(net, test_review_pos, seq_length)